### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import pandas as pd

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_df = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [19]:
#variable calculations
school_count = len(school_df["School ID"].unique())
student_count = (school_df["size"].unique()).sum()
total_budget = (school_df["budget"].unique()).sum()
avg_math = school_df["math_score"].mean()
avg_read = school_df["reading_score"].mean()
pass_math_bool = school_df["math_score"] >= 70
pass_read_bool = school_df["reading_score"] >= 70
pass_math = (pass_math_bool == True).sum()/student_count
pass_read = (pass_read_bool == True).sum()/student_count
overall_pass = (pass_math + pass_read) / 2



In [20]:
#creating dataframe
district_summary = pd.DataFrame({"Total Schools": [school_count],
                                "Total Students": [student_count],
                                "Total Budget": [total_budget],
                                "Average Math Score": [avg_math],
                                "Average Reading Score": [avg_read],
                                "% Passing Math": [pass_math],
                                "% Passing Reading": [pass_read],
                                "% Overall Passing Rate": [overall_pass]})

district_summary["Total Students"] = district_summary["Total Students"].astype(float).map("{:,.0f}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].astype(float).map("${:,.2f}".format)
district_summary["% Passing Math"] = district_summary["% Passing Math"].astype(float).map("{:,.6%}".format)
district_summary["% Passing Reading"] = district_summary["% Passing Reading"].astype(float).map("{:,.6%}".format)
district_summary["% Overall Passing Rate"] = district_summary["% Overall Passing Rate"].astype(float).map("{:,.6%}".format)

district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853%,85.805463%,80.393158%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [5]:
grouped_schools = school_df.groupby("school_name")

# add data that's easier to get from school_data
school_summary = pd.DataFrame({"School Name":school_data["school_name"], "School Type":school_data["type"],
                             "Total Budget":school_data["budget"]})
school_summary = school_summary.set_index("School Name")
school_summary = school_summary.sort_values("School Name")

# do calculations on data from groupby object
school_summary["Total Students"] = grouped_schools["Student ID"].count()
school_summary["Per Student Budget"] = school_summary["Total Budget"] / school_summary["Total Students"]
school_summary["Average Math Score"] = grouped_schools["math_score"].mean()
school_summary["Average Reading Score"] = grouped_schools["reading_score"].mean()

# calculate percentages of students passing using apply
school_summary["% Passing Math"] = round((grouped_schools.apply(lambda x: (x["math_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]), 8)
school_summary["% Passing Reading"] = round((grouped_schools.apply(lambda x: (x["reading_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]), 8)
school_summary["% Overall Passing Rate"] = round((school_summary["% Passing Math"] + 
                                               school_summary["% Passing Reading"]) / 2, 8)

school_summary = school_summary.sort_values("% Overall Passing Rate", ascending=False)

# need to switch total students and budget columns. by formatting on org_school.. we can use the values of school_summ... later
org_school_summary = school_summary[['School Type', 'Total Students', 'Total Budget', 'Per Student Budget',
       'Average Math Score', 'Average Reading Score', '% Passing Math',
       '% Passing Reading', '% Overall Passing Rate']]

org_school_summary["Total Students"] = org_school_summary["Total Students"].astype(float).map("{:,.0f}".format)
org_school_summary["Total Budget"] = org_school_summary["Total Budget"].astype(float).map("${:,.2f}".format)
org_school_summary["Per Student Budget"] = org_school_summary["Per Student Budget"].astype(float).map("${:,.2f}".format)
org_school_summary["% Passing Math"] = org_school_summary["% Passing Math"].astype(float).map("{:,.6%}".format)
org_school_summary["% Passing Reading"] = org_school_summary["% Passing Reading"].astype(float).map("{:,.6%}".format)
org_school_summary["% Overall Passing Rate"] = org_school_summary["% Overall Passing Rate"].astype(float).map("{:,.6%}".format)

org_school_summary.head()


,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,"1,858","$1,081,356.00",$582.00,83.061895,83.975780,94.133477%,97.039828%,95.586652%
Thomas High School,Charter,"1,635","$1,043,130.00",$638.00,83.418349,83.848930,93.272171%,97.308869%,95.290520%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595%,95.945946%,95.270271%
Griffin High School,Charter,"1,468","$917,500.00",$625.00,83.351499,83.816757,93.392371%,97.138965%,95.265668%
Wilson High School,Charter,"2,283","$1,319,574.00",$578.00,83.274201,83.989488,93.867718%,96.539641%,95.203680%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [6]:
org_school_summary = org_school_summary.sort_values("% Overall Passing Rate", ascending=True)

org_school_summary.head()

,School Type,Total Students,Total Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,"3,999","$2,547,363.00",$637.00,76.842711,80.744686,66.366592%,80.220055%,73.293324%
Figueroa High School,District,"2,949","$1,884,411.00",$639.00,76.711767,81.158020,65.988471%,80.739234%,73.363852%
Huang High School,District,"2,917","$1,910,635.00",$655.00,76.629414,81.182722,65.683922%,81.316421%,73.500172%
Johnson High School,District,"4,761","$3,094,650.00",$650.00,77.072464,80.966394,66.057551%,81.222432%,73.639992%
Ford High School,District,"2,739","$1,763,916.00",$644.00,77.102592,80.746258,68.309602%,79.299014%,73.804308%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [7]:
ninth = school_df.loc[school_df["grade"] == "9th"]
tenth = school_df.loc[school_df["grade"] == "10th"]
eleventh = school_df.loc[school_df["grade"] == "11th"]
twelfth = school_df.loc[school_df["grade"] == "12th"]

math_ninth = ninth.groupby("school_name")["math_score"].mean()
math_tenth = tenth.groupby("school_name")["math_score"].mean()
math_eleventh = eleventh.groupby("school_name")["math_score"].mean()
math_twelfth = twelfth.groupby("school_name")["math_score"].mean()

math_grades = pd.DataFrame({"9th Grade":math_ninth, "10th Grade":math_tenth, 
                             "11th Grade":math_eleventh, "12th Grade":math_twelfth})

math_grades = round(math_grades, 6)

math_grades

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [8]:
reading_ninth = ninth.groupby("school_name")["reading_score"].mean()
reading_tenth = tenth.groupby("school_name")["reading_score"].mean()
reading_eleventh = eleventh.groupby("school_name")["reading_score"].mean()
reading_twelfth = twelfth.groupby("school_name")["reading_score"].mean()

reading_grades = pd.DataFrame({"9th Grade":reading_ninth, "10th Grade":reading_tenth, 
                             "11th Grade":reading_eleventh, "12th Grade":reading_twelfth})

reading_grades = round(reading_grades, 6)

reading_grades

,9th Grade,10th Grade,11th Grade,12th Grade
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [9]:
#Multiply by 100 to get the correct formatting
school_summary["% Passing Math"] = round((grouped_schools.apply(lambda x: (x["math_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]) * 100, 8)
school_summary["% Passing Reading"] = round((grouped_schools.apply(lambda x: (x["reading_score"] >= 70).sum()) / 
                                          school_summary["Total Students"]) * 100, 8)
school_summary["% Overall Passing Rate"] = round((school_summary["% Passing Math"] + 
                                               school_summary["% Passing Reading"]) / 2, 8)

In [10]:
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [11]:
school_summary["Spending Ranges (Per Student)"] = pd.cut(school_summary["Per Student Budget"], spending_bins, labels=group_names)

spending_groups = school_summary.loc[:, ["Spending Ranges (Per Student)","Average Math Score", "Average Reading Score",
                                     "% Passing Math", "% Passing Reading",
                                     "% Overall Passing Rate"]].groupby("Spending Ranges (Per Student)")
spending_groups.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,95.035486
$585-615,83.599686,83.885211,94.230858,95.900287,95.065572
$615-645,79.079225,81.891436,75.668212,86.106569,80.887391
$645-675,76.997210,81.027843,66.164813,81.133951,73.649382


## Scores by School Size

* Perform the same operations as above, based on school size.

In [12]:
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [13]:
school_summary["School Size"] = pd.cut(school_summary["Total Students"], size_bins, labels=group_names)

size_groups = school_summary.loc[:, ["School Size","Average Math Score", "Average Reading Score",
                                     "% Passing Math", "% Passing Reading",
                                     "% Overall Passing Rate"]].groupby("School Size")

size_groups.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,94.824831
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,95.195187
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,76.364998


## Scores by School Type

* Perform the same operations as above, based on school type.

In [14]:
type_groups = school_summary.loc[:, ["School Type", "Average Math Score", "Average Reading Score",
                                     "% Passing Math", "% Passing Reading",
                                     "% Overall Passing Rate"]].groupby("School Type")

type_groups.mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,95.103660
District,76.956733,80.966636,66.548453,80.799062,73.673757
